In [1]:
from pyspark.sql import SparkSession

# pip install pykafka

# Kafka主题名称
kafka_topic_name = "songTopic"
# Kafka地址
kafka_bootstrap_servers = "node120:9092"

spark = SparkSession\
    .builder\
    .appName("streaming")\
    .master("local[4]")\
    .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

In [3]:
songs_df = spark\
    .readStream\
    .format("kafka")\
    .option("kafka.bootstrap.servers", kafka_bootstrap_servers)\
    .option("subscribe", kafka_topic_name)\
    .option("startingOffsets", "latest")\
    .load()

In [8]:
from pyspark.sql.functions import col, from_csv

songs_df1 = songs_df.selectExpr("CAST(value AS STRING)", "timestamp")
songs_schema_string = "order_id INT, id STRING, name STRING, popularity INT, duration_ms DOUBLE, explicit INT, "\
                      + "artists STRING, id_artists STRING, release_date STRING, "\
                      + "danceability DOUBLE,"\
                      + "energy DOUBLE, key INT, loudness DOUBLE, "\
                      + "mode INT,"\
                      + "speechiness DOUBLE,"\
                      + "acousticness DOUBLE, instrumentalness DOUBLE, liveness DOUBLE, "\
                      + "valence DOUBLE, tempo DOUBLE, time_signature DOUBLE"

songs_df2 = songs_df1\
    .select(from_csv(col("value"), songs_schema_string)\
    .alias("song"), "timestamp")

songs_df3 = songs_df2.select("song.*", "timestamp")
songs_df3.createOrReplaceTempView("song_find")
song_find_text = spark.sql("select count(1) from song_find")

In [ ]:
# 暂时先输入到控制台上
songs_agg_write_stream = song_find_text\
    .writeStream\
    .trigger(processingTime="5 seconds")\
    .outputMode("complete")\
    .option("truncate", "false")\
    .format("console")\
    .start()  # truncate 截断

songs_agg_write_stream.awaitTermination()